In [12]:
import pandas as pd
import numpy as np
import gdal

In [1]:
site_number = 1

In [7]:
sites = pd.read_csv("/DATA/These/Projects/modflops/docker-simulation/modflow/data/study_sites.txt", sep=',', header=0, index_col=0)
coordinates = sites._get_values[site_number,1:5]
print(coordinates)

[ 360487.5  374962.5 6868837.5 6882487.5]


In [13]:
def get_clip_dem(coord):
    r_dem = "/DATA/These/Projects/modflops/docker-simulation/modflow/data/MNT_TOPO_BATH_75m.tif"
    ulY, lrY, ulX, lrX, clip_dem_x, clip_dem_y = get_model_size(coord)
    dem = gdal.Open(r_dem)
    dem_geot = dem.GetGeoTransform()
    dem_data = dem.GetRasterBand(1).ReadAsArray()
    clip_dem = dem_data[ulY:lrY, ulX:lrX]
    return dem_geot, clip_dem_x, clip_dem_y, clip_dem

def get_model_size(coord):
    r_dem = "/DATA/These/Projects/modflops/docker-simulation/modflow/data/MNT_TOPO_BATH_75m.tif"
    xmin = coord[0]
    xmax = coord[1]
    ymin = coord[2]
    ymax = coord[3]
    dem = gdal.Open(r_dem)
    dem_geot = dem.GetGeoTransform()
    dem_Xpos = np.ones((dem.RasterXSize))
    dem_Ypos = np.ones((dem.RasterYSize))
    for i in range(0, dem.RasterYSize):
        yp = dem_geot[3] + (dem_geot[5] * i)
        dem_Ypos[i] = yp
    for j in range(0, dem.RasterXSize):
        xp = dem_geot[0] + (dem_geot[1] * j)
        dem_Xpos[j] = xp
    ulX = (np.abs(dem_Xpos - xmin)).argmin()
    lrX = (np.abs(dem_Xpos - xmax)).argmin()
    ulY = (np.abs(dem_Ypos - ymax)).argmin()
    lrY = (np.abs(dem_Ypos - ymin)).argmin()
    clip_dem_x = dem_Xpos[ulX:lrX]
    clip_dem_y = dem_Ypos[ulY:lrY]
    return ulY, lrY, ulX, lrX, clip_dem_x, clip_dem_y

In [14]:
geot, geotx, geoty, demData = get_clip_dem(coordinates)

In [20]:
print(demData.shape)

(182, 193)
